# Enterprise Excel Writer: Features Demo

This notebook demonstrates the full capabilities of the `EnterpriseExcelWriter` library. It replicates the logic used in our automated testing scripts but breaks it down step-by-step for clarity.

**Goal:** Generate a multi-tab, formatted Excel report for "Wales External" stakeholders.

**Key Features Covered:**
1. **Configuration:** Loading specific color schemes and titles from a database.
2. **Column Mapping:** Automating the renaming of technical columns (e.g., `total_revenue` -> `Revenue £`).
3. **Visuals:** Adding Logos, Sparklines, Charts, and Conditional Formatting.
4. **Structure:** Auto-generating a Table of Contents.

### 1. Setup & Imports
We import standard libraries and append the system path to locate our custom `src` folder. 

> **Note:** In a real Fabric environment, this path setup might look slightly different depending on where you store your library files (e.g., OneLake).

In [1]:
import sys
import os
import pandas as pd
import sqlite3
import numpy as np

# Add the 'src' directory to the path so we can import the wrapper class
# Assumes this notebook is in the 'notebooks/' folder and source is in 'src/'
sys.path.append(os.path.abspath(os.path.join(os.getcwd(), '..', 'src')))

from enterprise_writer import EnterpriseExcelWriter
from query_library import fGetRegionalSales, fGetDataDictionary
from config_provider import fGetReportConfig

### 2. Configuration & Data Retrieval
Instead of hardcoding colors and titles, we fetch a **Profile** from our configuration table. 
- **Profile:** `Wales_External`
- **Effect:** This will automatically set the primary color to Dragon Red (`#D30731`) and add the `CYMRU:` prefix to titles.

We also fetch the raw sales data and the data dictionary used for column renaming.

In [2]:
# 1. Fetch Configuration
vProfile = 'Wales_External'
vConfig = fGetReportConfig(vProfile) 

print(f"Loaded Config for {vProfile}")
print(f"Primary Color: {vConfig.get('primary_color')}")

# 2. Fetch Data
dfSales = fGetRegionalSales()
dfDict = fGetDataDictionary()

Loaded Config for Wales_External
Primary Color: #D30731


### 3. Initialize Report & Apply Branding
Here we start the Excel writer. 

**Crucial Step - Column Mapping:** 
We call `fSetColumnMapping(dfDict)`. This tells the writer to look at our dictionary table. Whenever it sees a technical name like `efficiency_rate`, it will automatically write **"Efficiency Score"** in the Excel header.

**Adding a Logo:**
We check if the config has a logo path. For this test to work, ensure you have an image file at the specified path (e.g., `notebooks/assets/logo.png`).

In [4]:
vFilename = "Wales_Performance_Report.xlsx"

# Initialize with the Config object (this sets the Theme Color automatically)
vReport = EnterpriseExcelWriter(vFilename, vConfig=vConfig)

# Apply Global Column Renaming Rule
vReport.fSetColumnMapping(dfDict)

# Add Logo (Top Left is default)
# Ensure 'assets/logo.png' exists relative to this notebook
# If using the config path, it would be: vReport.fAddLogo(vConfig.get('logo_path'))
#vReport.fAddLogo("assets/logo.png", vWidthScale=0.5)

### 4. Tab 1: Executive Summary
We build the main dashboard tab.

**Steps:**
1.  **Title:** Uses the prefix from the config (`CYMRU: Performance Report`).
2.  **KPIs:** `fAddKpiRow` creates the high-level metric cards.
3.  **Data Table:** `fWriteDataframe` writes the sales data. `vAddTotals=True` automatically calculates sums for numeric columns at the bottom.
4.  **Conditional Formatting:** We highlight any region where Efficiency is < 90% in light red.

In [5]:
# 1. Dynamic Title
vTitle = f"{vConfig.get('title_prefix', '')}Performance Report"
vReport.fAddTitle(vTitle)

# 2. KPI Cards
vReport.fAddKpiRow({
    'Total Revenue': '$1.2M', 
    'Year Growth': '5%'
})

# 3. Main Data Table with Totals Row
vReport.fWriteDataframe(dfSales, vAddTotals=True)

# 4. Conditional Formatting
# Note: We refer to the dataframe column name 'efficiency_rate', not the display name
vReport.fAddConditionalFormat('efficiency_rate', 'cell', {'criteria': '<', 'value': 0.90})

### 5. Adding Visuals (Charts & Sparklines)
Enterprise reports need visuals. 

- **Sparklines:** We simulate 12 months of trend data and add it as a "Trend" column next to the table.
- **Charts:** We add a native Excel Column Chart comparing Revenue by Region.

In [6]:
# 1. Sparklines (Simulated Data)
vTrends = np.random.randint(100, 200, size=(len(dfSales), 12)).tolist()
vReport.fAddSparklines(vTrends, vTitle="12-Month Trend")

# 2. Excel Chart
vReport.fAddChart(
    vTitle="Regional Revenue", 
    vType="column", 
    vXAxisCol="region_name", 
    vYAxisCols=["total_revenue"]
)

### 6. Tab 2: Appendix & TOC
We create a new sheet for the Data Dictionary. The `fAddDataDictionary` method is specialized to handle long text descriptions by wrapping text and merging cells.

Finally, `fGenerateTOC` creates a hyperlinked Table of Contents at the start of the workbook.

In [7]:
# New Sheet Context
vReport.fNewSheet("Appendix", "Detailed definitions")

# Specialized Writer
vReport.fAddDataDictionary(dfDict)

# Generate Table of Contents
vReport.fGenerateTOC()

# Save File
vReport.fClose()

print("Report Generation Complete.")

File saved: Wales_Performance_Report.xlsx
Report Generation Complete.
